<a href="https://colab.research.google.com/github/happyendingddd/CRISPR_Cas9_gRNA_design/blob/main/CNN_on_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
import tensorflow as tf

import pandas as pd
import numpy as np
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [212]:
# 数据导入1
file_path='/content/hela_binary.csv'
data_read=pd.read_csv(file_path,sep=',')
guideSeq=np.array(data_read['sgRNA'])
guideSeq=guideSeq.reshape((guideSeq.shape[0],1))
labels=np.array(data_read['Normalized_efficacy'])
print(guideSeq.shape)

(8101, 1)


In [213]:
def read_gdata(fp,num_read):
  with open(fp, "r") as f:
    list1=[]
    i=0
    for line in f.readlines():
      i+=1
      if i<=num_read:
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        list1.append(line)
  arr=np.array(list1)
  seq_g=arr.reshape((arr.shape[0],1))
  return seq_g

In [214]:
FP='/content/generated_seq_hela_positive.txt'
FN='/content/generated_seq_hela_negetive.txt'
num_read=int(guideSeq.shape[0]/2)
seq_gP=read_gdata(FP,num_read)
seq_gN=read_gdata(FN,num_read)
seq_g=np.vstack([seq_gP,seq_gN])
labels_gP=np.zeros(seq_gP.shape[0])+1
labels_gN=np.zeros(seq_gN.shape[0])
labels_g=np.r_[labels_gP,labels_gN]
labels_all=np.r_[labels,labels_g]

In [215]:
seq_all=np.vstack([guideSeq,seq_g])
print(seq_all.shape)

(16201, 1)


In [216]:
seq_all=seq_all.reshape(seq_all.shape[0])
print(seq_all.shape)

(16201,)


In [217]:
#one-hot编码
ntmap = {'A': (1, 0, 0, 0),
         'C': (0, 1, 0, 0),
         'G': (0, 0, 1, 0),
         'T': (0, 0, 0, 1)
         }

def get_seqcode(seq):
    return list(map(lambda c: ntmap[c], seq))

def oneHotcoding(Seq):
    n=0
    for seq in Seq:
        if n==0:
            SeqcodeL=[]
        seqcode=get_seqcode(seq)
        n+=1
        SeqcodeL.append(seqcode)
        seqcode=[]
        SeqcodeA=np.array(SeqcodeL)
    return SeqcodeA

In [ ]:
guidecode=oneHotcoding(seq_all)

In [ ]:
guidecode.shape

In [ ]:
seqdata_all=guidecode
print(seqdata_all.shape)

In [ ]:
from sklearn.model_selection import train_test_split
train_seq,test_seq,train_lab,test_lab=train_test_split(seqdata_all,labels_all,train_size = 0.75,random_state=14,stratify=labels_all)

In [ ]:
train_seq.shape

In [ ]:
def CNN_model():
  model = tf.keras.Sequential()
  
  model.add(layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(23,4)))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(64, 3,padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(64, 3,padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(2))

  return model

In [ ]:
model=CNN_model()
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_seq, train_lab, epochs=50, batch_size=16,
                    validation_data=(test_seq, test_lab))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_seq,  test_lab, verbose=2)

In [ ]:
print(test_acc)
#before
#hl60=0.9614643454551697
#hela=0.8524185419082642
#hct116=0.8669811487197876
#hek293t=0.948630154132843
#after
#hl60=0.9816955924034119

In [ ]:
# 保存模型
model.save('/content/model_trained.h5') 


In [ ]:
# 加载模型，同时加载了模型的结构、权重等信息
loaded_model = tf.keras.models.load_model('/content/model_trained.h5')